In [2]:
import contextlib
import os
import re
import sys

import linearmodels as lm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns
import tabulate

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.selectors as sl
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [40]:
df = hd.read_sample("X77")

Time for read_sample    : 19.83 seconds


In [44]:
df.user_id.nunique()

599

In [42]:
sl.no_missing_months(df).user_id.nunique()

595

In [37]:
df.set_index("date").groupby("user_id").resample("m").id.count().index

MultiIndex([(   177, '2012-01-31'),
            (   177, '2012-02-29'),
            (   177, '2012-03-31'),
            (   177, '2012-04-30'),
            (   177, '2012-05-31'),
            (   177, '2012-06-30'),
            (   177, '2012-07-31'),
            (   177, '2012-08-31'),
            (   177, '2012-09-30'),
            (   177, '2012-10-31'),
            ...
            (589277, '2019-10-31'),
            (589277, '2019-11-30'),
            (589277, '2019-12-31'),
            (589277, '2020-01-31'),
            (589277, '2020-02-29'),
            (589277, '2020-03-31'),
            (589277, '2020-04-30'),
            (589277, '2020-05-31'),
            (589277, '2020-06-30'),
            (589277, '2020-07-31')],
           names=['user_id', 'date'], length=17862)

In [39]:
month = pd.Grouper(key="date", freq="m")
df.groupby(["user_id", month]).id.count().index

MultiIndex([(   177, '2012-01-31'),
            (   177, '2012-02-29'),
            (   177, '2012-03-31'),
            (   177, '2012-04-30'),
            (   177, '2012-05-31'),
            (   177, '2012-06-30'),
            (   177, '2012-07-31'),
            (   177, '2012-08-31'),
            (   177, '2012-09-30'),
            (   177, '2012-10-31'),
            ...
            (589277, '2019-10-31'),
            (589277, '2019-11-30'),
            (589277, '2019-12-31'),
            (589277, '2020-01-31'),
            (589277, '2020-02-29'),
            (589277, '2020-03-31'),
            (589277, '2020-04-30'),
            (589277, '2020-05-31'),
            (589277, '2020-06-30'),
            (589277, '2020-07-31')],
           names=['user_id', 'date'], length=17834)